In [13]:
#@title [0] Keep Colab Alive</font>
#@markdown This cell runs a JS code that automatically presses the reconnect button in every 60 seconds to avoid inactivity due to idle timeout.

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected");
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!");
            },60000);
    }

 function clearRefresher() {
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

In [20]:
#@title [1] Mount Google Drive
clear_output()
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [14]:
#@title [2] Install DAT
#@markdown This notebook uses the **DAT architecture** by Zheng Chen  \
#@markdown [DAT on GitHub](https://github.com/zhengchen1999/DAT)
from IPython.display import clear_output
clear_output()
%cd /content
!git clone https://github.com/zhengchen1999/DAT.git
%cd /content/DAT
!echo Installed at /content/DAT

/content
fatal: destination path 'DAT' already exists and is not an empty directory.
/content/DAT
Installed at /content/DAT


In [15]:
#@title [3] Install others
!pip3 install -r requirements.txt
!pip3 install -q spandrel gdown

  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached lmdb-1.7.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached tb_nightly-2.21.0a20250926-py3-none-any.whl.metadata (1.9 kB)
ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torch==1.8.0


In [16]:
#@title [4] Install model
import ipywidgets as widgets
clear_output()
%mkdir /content/models
%cd /content/models
!gdown -q 1RKPvYnYp4xy_PLnHr3le92CarIgr1LbN #Manga Ora
!gdown -q 1JLSbRRaaWSdndwcQP1n6xqwRteYxJr1R #DAT light x2
print("Model(s) downloaded.")

mkdir: cannot create directory ‘/content/models’: File exists
/content/models
Model(s) downloaded.


In [21]:
#@title [6] Set input/output folder

#Uncomment if use colab drive
#input_folder = '/content/input'
#output_folder = '/content/output'

#Uncomment if mount Google drive
input_folder = '/content/drive/MyDrive/Nnn/input'
output_folder = '/content/drive/MyDrive/Nnn/output'


In [22]:
#@title [7] Actual upscaling work (specifically for manga)
import os
import cv2
import torch
from spandrel import ModelLoader, ImageModelDescriptor
from pathlib import Path

#get all images's path
images = [p for p in Path(input_folder).iterdir()
          if p.suffix.lower() in [".png", ".jpg", ".jpeg"]]

#Load model
Model_Name = "2x_Manga_Ora.pth" #Mofify this if you use other model
model_path = f"/content/models/{Model_Name}"

# Check if model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
else:
    model = ModelLoader().load_from_file(model_path)
    assert isinstance(model, ImageModelDescriptor)

    if torch.cuda.is_available():
        model.cuda().eval()
        device = 'cuda'
    else:
        model.eval()
        device = 'cpu'
        print("CUDA not available, using CPU.")


    for img_path in images:
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not load image {img_path}")
            continue

        # Convert image to RGB (spandrel requires RGB)
        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #convert to tensor
        tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float() / 255

        tensor = tensor.to(device)

        with torch.no_grad():
            #inference
            result = model(tensor)

            # move result back to CPU before converting to numpy
            result_image = (result.detach().cpu().squeeze().permute(1, 2, 0).numpy().clip(0, 1) * 255).round().astype(
                'uint8')

            # Convert result image to grayscale
            result_image_gray = cv2.cvtColor(result_image, cv2.COLOR_RGB2GRAY)

            filename = os.path.basename(img_path)
            output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".png")

            # Create output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)
            cv2.imwrite(output_path, result_image_gray)
            print(f'Upscaled and saved {filename} as grayscale PNG')
    print("DONE")

Upscaled and saved 4.jpg as grayscale PNG
DONE


In [ ]:
#@title [Bonus] General upscale (not grayscale)
import os
import cv2
import torch
from spandrel import ModelLoader, ImageModelDescriptor
from pathlib import Path

#get all images's path
images = [p for p in Path(input_folder).iterdir()
          if p.suffix.lower() in [".png", ".jpg", ".jpeg"]]

#Load model
Model_Name = "DAT_light_x2.pth" #Mofify this if you use other model
model_path = f"/content/models/{Model_Name}"

# Check if model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
else:
    model = ModelLoader().load_from_file(model_path)
    assert isinstance(model, ImageModelDescriptor)

    if torch.cuda.is_available():
        model.cuda().eval()
        device = 'cuda'
    else:
        model.eval()
        device = 'cpu'
        print("CUDA not available, using CPU.")


    for img_path in images:
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not load image {img_path}")
            continue

        # Convert image to RGB (spandrel requires RGB)
        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #convert to tensor
        tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float() / 255

        tensor = tensor.to(device)

        with torch.no_grad():
            #inference
            result = model(tensor)

            # move result back to CPU before converting to numpy
            result_image = (result.detach().cpu().squeeze().permute(1, 2, 0).numpy().clip(0, 1) * 255).round().astype(
                'uint8')

            filename = os.path.basename(img_path)
            output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".png")

            # Create output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)
            cv2.imwrite(output_path, cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))
            print(f'Upscaled and saved {filename} as PNG')
    print("DONE")

In [ ]:
#@title [Optional] Zip output folder for download if used colab's drive
import shutil
import os

output_folder = '/content/output'
zip_file_name = '/content/output.zip'

# Create the output folder if it doesn't exist (in case it was deleted)
os.makedirs(output_folder, exist_ok=True)

# Zip the contents of the output folder
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', output_folder)

print(f'Successfully zipped {output_folder} to {zip_file_name}')

Successfully zipped /content/output to /content/output.zip
